In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy import stats
import statsmodels.api as sm
import tqdm


In [2]:
predictive_processing_dataset = pd.read_excel("All_Features_dataset.xlsx")
scales = pd.read_excel("filled_scales_BEBRASK_RETOS.xlsx")


In [3]:
scales.drop(["SUBJECT_CODE","Age"],axis=1,inplace=True)

common_values = scales['EPRIME_CODE'].unique()
predictive_processing_dataset = predictive_processing_dataset[predictive_processing_dataset['Subject'].isin(common_values)]


In [4]:
metrics_columns = scales.drop("EPRIME_CODE",axis=1).columns.values
predictive_processing_columns = predictive_processing_dataset.drop("Subject",axis=1).columns.values

In [5]:
feature_scales = pd.merge(left = predictive_processing_dataset,right=scales, left_on="Subject",right_on="EPRIME_CODE").drop(["EPRIME_CODE","Subject"],axis=1)

In [48]:
values = list(range(5, 96, 5))

In [49]:
feature_scales[predictive_processing_columns]

,Bin,Cor_Pred_Like,Cor_Pred_Like_Match,Cor_Pred_Like_Match_Fear,Cor_Pred_Like_Match_Happy,Cor_Pred_Like_Match_Sad,Cor_Pred_Like_No_Match,Cor_Pred_Like_No_Match_Fear,Cor_Pred_Like_No_Match_Happy,Cor_Pred_Like_No_Match_Sad,...,Mean_Rating0_No_Match_Happy,Mean_Rating0_No_Match_Sad,Trend_Match,Trend_Match_Fear,Trend_Match_Happy,Trend_Match_Sad,Trend_No_Match,Trend_No_Match_Fear,Trend_No_Match_Happy,Trend_No_Match_Sad
0,Above,0.113495,0.209359,0.196722,-2.500000e-01,0.342381,0.103807,-0.188982,0.542326,-0.061430,...,1.500000,2.166667,-0.004884,0.050000,-0.066667,0.016667,-0.045408,-4.000000e-01,-4.285714e-01,2.000000e-01
1,Above,0.214781,0.400320,0.395285,4.902903e-01,0.059761,-0.070711,0.158114,-0.632456,0.000000,...,1.333333,1.833333,-0.015263,-0.066667,-0.100000,0.050000,0.036120,2.857143e-01,-1.142857e-01,8.571429e-02
2,Above,0.250345,0.432551,0.307692,-2.294157e-01,0.230940,0.139859,-0.307148,0.800000,0.000000,...,1.166667,1.500000,0.012210,-0.033333,0.066667,0.116667,0.003096,8.571429e-02,8.571429e-02,-1.428571e-01
3,Below,-0.230171,-0.006600,-0.280000,1.409710e-02,0.000000,-0.585369,-0.131306,0.000000,-0.542326,...,1.000000,2.666667,0.033578,-0.016667,0.150000,0.183333,-0.014448,1.166434e-16,-3.812839e-17,-4.571429e-01
4,Above,0.176201,0.334275,-0.762493,5.345225e-01,0.162221,-0.194348,-0.867722,0.415227,-0.387298,...,1.166667,1.333333,0.039683,0.200000,-0.116667,0.100000,0.005160,-2.857143e-01,-2.857143e-02,1.714286e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Above,0.255697,0.281127,0.463586,9.433333e-02,-0.036564,0.260715,-0.316228,0.727607,0.342997,...,1.500000,1.333333,0.012821,0.016667,0.150000,-0.116667,0.052632,2.857143e-01,1.428571e-01,1.142857e-01
145,Above,0.608424,0.418441,0.304997,2.467162e-17,-0.125000,0.346488,0.242536,-0.108465,0.542326,...,1.166667,1.333333,0.014042,-0.016667,-0.050000,-0.066667,0.024768,2.571429e-01,1.428571e-01,-2.857143e-01
146,Above,0.440557,0.508058,0.548795,7.581298e-01,0.346423,0.000000,-0.242536,0.000000,-0.121566,...,1.000000,1.833333,-0.012210,0.033333,-0.200000,-0.100000,0.031992,-5.714286e-02,-3.812839e-17,8.571429e-02
147,Above,0.376099,0.486755,0.202031,1.000000e+00,0.000000,-0.258969,-0.542326,0.612372,-0.800000,...,1.333333,1.666667,-0.024420,-0.200000,0.083333,-0.100000,0.021672,-2.857143e-02,1.714286e-01,-5.714286e-02


In [8]:

dataframes = {}
all_dictionary = {}
for value in values:
    
# Assuming feature_scales is your DataFrame with features as columns and metrics as rows
    dictionary_distribution = {}

    for metric in metrics_columns:
        dictionary_distribution[metric] = {}
        
        
        for feature in predictive_processing_columns:
            percentile = np.percentile(feature_scales[feature],value)
            if value < 50:
                above_values = feature_scales[feature_scales[feature] > percentile][metric]  
                below_values = feature_scales[feature_scales[feature] <= percentile][metric]  
            else:
                above_values = feature_scales[feature_scales[feature] >= percentile][metric]  
                below_values = feature_scales[feature_scales[feature] < percentile][metric]  

            u_stat, p_value = stats.mannwhitneyu(above_values, below_values, alternative='two-sided')    
            # Calculate means
            high_mean = np.mean(above_values)
            low_mean = np.mean(below_values)
    
            dictionary_distribution[metric][feature] = {'high_mean': high_mean, 'size_high': len(above_values),
                                                       'low_mean': low_mean, 'size_low': len(below_values),
                                                       'p_value': p_value}

    all_dictionary[value] = dictionary_distribution
    df = pd.DataFrame.from_dict({(metric, feature): values
                                 for metric, features in dictionary_distribution.items()
                                 for feature, values in features.items()}, orient='index')

    # Add this DataFrame to the main dictionary of DataFrames
    dataframes[value] = df

In [9]:
p_value_comparison = {}

# Iterate through each metric
for metric in metrics_columns:
    # Initialize a list to store mean p-values per percentile value
    mean_p_values = []

    for percentile in values:
        # Extract p-values for the current percentile dataset
        p_values = dataframes[percentile].xs(metric, level=0)['p_value']
        mean_p_values.append((percentile, p_values.mean()))

    # Find the percentile dataset with the lowest mean p-value
    lowest_p_value_percentile = min(mean_p_values, key=lambda x: x[1])

    # Store the results in the dictionary
    p_value_comparison[metric] = {
        'Percentile with Lowest Mean P-Value': lowest_p_value_percentile[0],
        'Mean P-Value': lowest_p_value_percentile[1]
    }

# Display the comparison results
for metric, result in p_value_comparison.items():
    print(f"\nMetric: {metric}")
    print(f"Percentile with Lowest Mean P-Value: {result['Percentile with Lowest Mean P-Value']}")
    print(f"Mean P-Value: {result['Mean P-Value']}")


Metric: PA
Percentile with Lowest Mean P-Value: 45
Mean P-Value: 0.27726910382660724

Metric: NA.
Percentile with Lowest Mean P-Value: 30
Mean P-Value: 0.40954996442424746

Metric: ERQ_CR
Percentile with Lowest Mean P-Value: 35
Mean P-Value: 0.3501655553214898

Metric: ERQ_ES
Percentile with Lowest Mean P-Value: 85
Mean P-Value: 0.4416079683513129

Metric: UPPSP_NU
Percentile with Lowest Mean P-Value: 20
Mean P-Value: 0.45631176180307975

Metric: UPPSP_PU
Percentile with Lowest Mean P-Value: 70
Mean P-Value: 0.4190106496840595

Metric: UPPSP_SS
Percentile with Lowest Mean P-Value: 30
Mean P-Value: 0.42564935339939064

Metric: UPPSP_PMD
Percentile with Lowest Mean P-Value: 70
Mean P-Value: 0.4214795703957438

Metric: UPPSP_PSV
Percentile with Lowest Mean P-Value: 85
Mean P-Value: 0.3200573960635867

Metric: BIS
Percentile with Lowest Mean P-Value: 90
Mean P-Value: 0.3898690414559161

Metric: BAS_RR
Percentile with Lowest Mean P-Value: 40
Mean P-Value: 0.40403225362996337

Metric: BAS_D

In [10]:
# Initialize a dictionary to store the top three significant counts per metric
top_significant_counts_per_metric = {metric: [] for metric in metrics_columns}

# Iterate through each metric to determine the top three percentiles with the most significant features
for metric in metrics_columns:
    # Create a list to hold the counts for each percentile for this metric
    metric_counts = []

    for percentile, df in dataframes.items():
        # Filter only rows corresponding to the current metric
        p_values = df.xs(metric, level=0)['p_value']

        # Count the number of features with p-values below 0.05
        significant_count = (p_values < 0.05).sum()

        # Add the count to the list
        metric_counts.append((percentile, significant_count))

    # Sort the list of counts in descending order by count
    metric_counts.sort(key=lambda x: x[1], reverse=True)

    # Keep only the top three counts
    top_significant_counts_per_metric[metric] = metric_counts[:3]

# Display the results
for metric, top_counts in top_significant_counts_per_metric.items():
    print(f"\nMetric: {metric}")
    for rank, (percentile, count) in enumerate(top_counts, 1):
        print(f"  {rank}- Highest: Percentile {percentile} with {count} features having significant p-values")




Metric: PA
  1- Highest: Percentile 45 with 13 features having significant p-values
  2- Highest: Percentile 75 with 12 features having significant p-values
  3- Highest: Percentile 30 with 11 features having significant p-values

Metric: NA.
  1- Highest: Percentile 20 with 5 features having significant p-values
  2- Highest: Percentile 30 with 4 features having significant p-values
  3- Highest: Percentile 45 with 4 features having significant p-values

Metric: ERQ_CR
  1- Highest: Percentile 35 with 9 features having significant p-values
  2- Highest: Percentile 55 with 8 features having significant p-values
  3- Highest: Percentile 25 with 7 features having significant p-values

Metric: ERQ_ES
  1- Highest: Percentile 10 with 4 features having significant p-values
  2- Highest: Percentile 75 with 4 features having significant p-values
  3- Highest: Percentile 5 with 3 features having significant p-values

Metric: UPPSP_NU
  1- Highest: Percentile 40 with 3 features having signific

In [11]:
# Initialize a dictionary to store counts of significant p-values per feature across all percentiles
significant_counts_aggregate = {feature: [] for feature in predictive_processing_columns}

# Iterate through each percentile and count the significant p-values per feature
for percentile in values:
    df = dataframes[percentile]
    
    # Group by feature only (ignoring the metric)
    significant_count = df['p_value'].groupby(level=1).apply(lambda x: (x < 0.05).sum())
    
    # Append the count for each feature to its corresponding list
    for feature, count in significant_count.items():
        significant_counts_aggregate[feature].append((percentile, count))

# Print the results per feature
for feature, counts in significant_counts_aggregate.items():
    print(f"\nFeature: {feature}")
    for percentile, count in counts:
        print(f"  Percentile {percentile}: {count} significant p-values")



Feature: Mean_Rating0
  Percentile 5: 1 significant p-values
  Percentile 10: 1 significant p-values
  Percentile 15: 1 significant p-values
  Percentile 20: 3 significant p-values
  Percentile 25: 1 significant p-values
  Percentile 30: 3 significant p-values
  Percentile 35: 0 significant p-values
  Percentile 40: 2 significant p-values
  Percentile 45: 3 significant p-values
  Percentile 50: 3 significant p-values
  Percentile 55: 3 significant p-values
  Percentile 60: 1 significant p-values
  Percentile 65: 1 significant p-values
  Percentile 70: 1 significant p-values
  Percentile 75: 1 significant p-values
  Percentile 80: 1 significant p-values
  Percentile 85: 0 significant p-values
  Percentile 90: 0 significant p-values
  Percentile 95: 0 significant p-values

Feature: Mean_Rating0_Match
  Percentile 5: 1 significant p-values
  Percentile 10: 0 significant p-values
  Percentile 15: 2 significant p-values
  Percentile 20: 5 significant p-values
  Percentile 25: 1 significant

In [12]:
# Initialize a dictionary to store the top three counts of significant p-values per feature
top_significant_counts_per_feature = {feature: [] for feature in predictive_processing_columns}

# Iterate through each percentile and count the significant p-values per feature
for percentile in values:
    df = dataframes[percentile]

    # Group by feature only (ignoring the metric)
    significant_count = df['p_value'].groupby(level=1).apply(lambda x: (x < 0.05).sum())

    # Append the count for each feature to its list
    for feature, count in significant_count.items():
        top_significant_counts_per_feature[feature].append((percentile, count))

# For each feature, sort the counts and keep only the top three
for feature, counts in top_significant_counts_per_feature.items():
    # Sort by the count (second element of the tuple) in descending order
    counts.sort(key=lambda x: x[1], reverse=True)
    # Keep only the top three
    top_significant_counts_per_feature[feature] = counts[:3]

# Print the results showing the top three counts for each feature
for feature, top_counts in top_significant_counts_per_feature.items():
    print(f"\nFeature: {feature}")
    for rank, (percentile, count) in enumerate(top_counts, 1):
        print(f"  {rank}- Highest: Percentile {percentile} with {count} significant p-values")



Feature: Mean_Rating0
  1- Highest: Percentile 20 with 3 significant p-values
  2- Highest: Percentile 30 with 3 significant p-values
  3- Highest: Percentile 45 with 3 significant p-values

Feature: Mean_Rating0_Match
  1- Highest: Percentile 20 with 5 significant p-values
  2- Highest: Percentile 45 with 4 significant p-values
  3- Highest: Percentile 50 with 4 significant p-values

Feature: Mean_Rating0_No_Match
  1- Highest: Percentile 10 with 3 significant p-values
  2- Highest: Percentile 15 with 3 significant p-values
  3- Highest: Percentile 20 with 3 significant p-values

Feature: Mean_Rating0_Match_Happy
  1- Highest: Percentile 45 with 4 significant p-values
  2- Highest: Percentile 55 with 4 significant p-values
  3- Highest: Percentile 60 with 4 significant p-values

Feature: Mean_Rating0_No_Match_Happy
  1- Highest: Percentile 65 with 5 significant p-values
  2- Highest: Percentile 70 with 5 significant p-values
  3- Highest: Percentile 75 with 5 significant p-values

Fe

In [13]:
# Initialize a list to store the total count of significant p-values for each percentile
total_significant_counts_per_percentile = []

# Iterate through each percentile and count the significant p-values
for percentile in values:
    df = dataframes[percentile]
    
    # Count the number of significant p-values across all features and metrics
    total_significant_count = (df['p_value'] < 0.05).sum()
    
    # Store the total count with its corresponding percentile
    total_significant_counts_per_percentile.append((percentile, total_significant_count))

# Sort the percentiles by the total count of significant p-values in descending order
sorted_significant_counts = sorted(total_significant_counts_per_percentile, key=lambda x: x[1], reverse=True)

# Print the sorted list
print("\nOrdered list of percentiles by significant p-values count (descending):")
for rank, (percentile, count) in enumerate(sorted_significant_counts, 1):
    print(f"{rank}. Percentile {percentile}: {count} significant p-values")




Ordered list of percentiles by significant p-values count (descending):
1. Percentile 90: 107 significant p-values
2. Percentile 45: 102 significant p-values
3. Percentile 30: 100 significant p-values
4. Percentile 80: 99 significant p-values
5. Percentile 95: 98 significant p-values
6. Percentile 40: 94 significant p-values
7. Percentile 75: 91 significant p-values
8. Percentile 35: 90 significant p-values
9. Percentile 85: 90 significant p-values
10. Percentile 50: 86 significant p-values
11. Percentile 60: 86 significant p-values
12. Percentile 70: 85 significant p-values
13. Percentile 55: 84 significant p-values
14. Percentile 20: 82 significant p-values
15. Percentile 25: 80 significant p-values
16. Percentile 65: 79 significant p-values
17. Percentile 15: 65 significant p-values
18. Percentile 5: 49 significant p-values
19. Percentile 10: 48 significant p-values


In [14]:
from collections import defaultdict

# Initialize a dictionary to store feature counts and percentiles for each metric
feature_counts_and_percentiles_per_metric = {
    metric: defaultdict(lambda: {"count": 0, "percentiles": []}) for metric in metrics_columns
}

# Iterate through each percentile DataFrame to count significant features per metric
for percentile, df in dataframes.items():
    for metric in metrics_columns:
        # Extract rows corresponding to the current metric
        metric_data = df.xs(metric, level=0)

        # Increment the count and record the percentile for features where the p-value is below 0.05
        for feature, row in metric_data.iterrows():
            if row['p_value'] < 0.05:
                feature_info = feature_counts_and_percentiles_per_metric[metric][feature]
                feature_info["count"] += 1
                feature_info["percentiles"].append(percentile)

# Determine the top 5 features for each metric
top_features_per_metric = {}

for metric, feature_data in feature_counts_and_percentiles_per_metric.items():
    # Sort features by their count in descending order and get the top 5
    sorted_features = sorted(feature_data.items(), key=lambda x: x[1]["count"], reverse=True)[:5]
    top_features_per_metric[metric] = sorted_features

# Print the results
for metric, top_features in top_features_per_metric.items():
    print(f"\nTop 5 features for metric {metric}:")
    for rank, (feature, data) in enumerate(top_features, 1):
        percentiles_str = ", ".join(map(str, sorted(data["percentiles"])))
        print(f"{rank}. Feature {feature}: {data['count']} significant appearances (percentiles: {percentiles_str})")



Top 5 features for metric PA:
1. Feature Mean_Rating0: 14 significant appearances (percentiles: 5, 10, 15, 20, 30, 40, 45, 50, 55, 60, 65, 70, 75, 80)
2. Feature Final_Value_No_Match_Sad: 14 significant appearances (percentiles: 5, 10, 15, 20, 25, 35, 40, 45, 50, 65, 70, 75, 85, 90)
3. Feature Mean_Rating0_Match_Sad: 13 significant appearances (percentiles: 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 75, 80, 85)
4. Feature Mean_Rating0_Match: 11 significant appearances (percentiles: 15, 20, 40, 45, 50, 55, 60, 65, 70, 75, 80)
5. Feature Final_Value_Match_Fear: 10 significant appearances (percentiles: 30, 55, 60, 65, 70, 75, 80, 85, 90, 95)

Top 5 features for metric NA.:
1. Feature Intercept_No_Match_Happy: 12 significant appearances (percentiles: 15, 20, 25, 30, 35, 40, 45, 60, 65, 70, 75, 80)
2. Feature Intercept_Match_Fear: 7 significant appearances (percentiles: 15, 20, 25, 30, 35, 45, 50)
3. Feature Trend_No_Match_Happy: 7 significant appearances (percentiles: 20, 25, 50, 55, 60, 65,

In [15]:
for i in range(1,len(values)):
    print(f'Overlap in {values[i]} percentil from {values[i-1]} percentil:')
    print(len(dataframes[values[i]][dataframes[values[i]]["p_value"]<0.05].index[dataframes[values[i]][dataframes[values[i]]["p_value"]<0.05].index.isin(dataframes[values[i-1]][dataframes[values[i-1]]["p_value"]<0.05].index)]))
    print(dataframes[values[i]][dataframes[values[i]]["p_value"]<0.05].index[dataframes[values[i]][dataframes[values[i]]["p_value"]<0.05].index.isin(dataframes[values[i-1]][dataframes[values[i-1]]["p_value"]<0.05].index)])

Overlap in 10 percentil from 5 percentil:
21
MultiIndex([(      'PA',                 'Mean_Rating0'),
            (      'PA', 'Cor_Pred_Like_No_Match_Happy'),
            (      'PA',     'Final_Value_No_Match_Sad'),
            (  'ERQ_ES',  'Mean_Rating0_No_Match_Happy'),
            ('UPPSP_PU',              'Trend_Match_Sad'),
            ('UPPSP_PU',        'Final_Value_Match_Sad'),
            ('UPPSP_SS',      'Mean_Rating0_Match_Fear'),
            ('UPPSP_SS',              'Trend_Match_Sad'),
            (  'BAS_RR',  'Mean_Rating0_No_Match_Happy'),
            (   'BAS_D',  'Mean_Rating0_No_Match_Happy'),
            (  'BAS_FS',              'Intercept_Match'),
            (  'BAS_FS',       'Final_Value_Match_Fear'),
            ( 'TEPS_CF',     'Cor_Pred_Like_Match_Fear'),
            ( 'TEPS_CF',     'Final_Value_No_Match_Sad'),
            (      'FS',     'Final_Value_No_Match_Sad'),
            (   'LOT_R',  'Mean_Rating0_No_Match_Happy'),
            ( 'RRQ_Ref',   

In [16]:
overlap_dict = {}

feature_overlap_count = {}

# Initialize the feature overlap count dictionary with metrics and empty dictionaries
for df in dataframes.values():
    for metric in df.index.levels[0]:  # Assuming MultiIndex with metrics at level 0
        if metric not in feature_overlap_count:
            feature_overlap_count[metric] = {}

for i in range(1, len(values)):
    prev_percentile = values[i-1]
    current_percentile = values[i]
    overlap_key = f"Overlap in {current_percentile} percentile from {prev_percentile} percentile"
    
    # Get significant features (indices where p_value < 0.05) for both current and previous percentiles
    significant_prev = dataframes[prev_percentile][dataframes[prev_percentile]["p_value"] < 0.05].index
    significant_current = dataframes[current_percentile][dataframes[current_percentile]["p_value"] < 0.05].index

    # Determine the overlap using set intersection
    overlap_indices = significant_current[significant_current.isin(significant_prev)]

    # Creating nested dictionary for storing metrics and their overlapping features
    metric_overlap = {}
    for (metric, feature) in overlap_indices:
        if metric not in metric_overlap:
            metric_overlap[metric] = []
        metric_overlap[metric].append(feature)
        if feature not in feature_overlap_count[metric]:
            feature_overlap_count[metric][feature] = 1
        else:
            feature_overlap_count[metric][feature] += 1

    # Store the overlap dictionary under the corresponding key
    overlap_dict[overlap_key] = metric_overlap

# Display the results of overlaps on consecutive percentiles
for key, metrics in overlap_dict.items():
    print("\n")
    print(key)
    for metric, features in metrics.items():
        print(f"  Metric: {metric}, Features: {features}")
        



Overlap in 10 percentile from 5 percentile
  Metric: PA, Features: ['Mean_Rating0', 'Cor_Pred_Like_No_Match_Happy', 'Final_Value_No_Match_Sad']
  Metric: ERQ_ES, Features: ['Mean_Rating0_No_Match_Happy']
  Metric: UPPSP_PU, Features: ['Trend_Match_Sad', 'Final_Value_Match_Sad']
  Metric: UPPSP_SS, Features: ['Mean_Rating0_Match_Fear', 'Trend_Match_Sad']
  Metric: BAS_RR, Features: ['Mean_Rating0_No_Match_Happy']
  Metric: BAS_D, Features: ['Mean_Rating0_No_Match_Happy']
  Metric: BAS_FS, Features: ['Intercept_Match', 'Final_Value_Match_Fear']
  Metric: TEPS_CF, Features: ['Cor_Pred_Like_Match_Fear', 'Final_Value_No_Match_Sad']
  Metric: FS, Features: ['Final_Value_No_Match_Sad']
  Metric: LOT_R, Features: ['Mean_Rating0_No_Match_Happy']
  Metric: RRQ_Ref, Features: ['Cor_Pred_Like_Match_Fear', 'Final_Value_No_Match']
  Metric: ASI_C, Features: ['Mean_Rating0_Match_Happy', 'Dif_Happy']
  Metric: SPQ, Features: ['Final_Value_No_Match_Sad']


Overlap in 15 percentile from 10 percentile


In [17]:
# Display the results of number of overlaps
for metric, features in feature_overlap_count.items():
    print("\n")
    print(f"Metric: {metric}")
    for feature, count in features.items():
        print(f"  Feature: {feature}, Overlap Count: {count}")



Metric: ASI_C
  Feature: Mean_Rating0_Match_Happy, Overlap Count: 4
  Feature: Dif_Happy, Overlap Count: 3
  Feature: Final_Value_No_Match_Happy, Overlap Count: 7
  Feature: Dif_Sad, Overlap Count: 4
  Feature: Dif_Match, Overlap Count: 1
  Feature: Final_Value_Match, Overlap Count: 1


Metric: ASI_P
  Feature: Mean_Rating0_Match_Happy, Overlap Count: 5
  Feature: Final_Value_Match_Happy, Overlap Count: 3
  Feature: Dif_Sad, Overlap Count: 2
  Feature: Dif_Match, Overlap Count: 5
  Feature: Final_Value_Match, Overlap Count: 1


Metric: ASI_S
  Feature: Trend_No_Match_Happy, Overlap Count: 4
  Feature: Final_Value_Match_Fear, Overlap Count: 2
  Feature: Cor_Pred_Like_No_Match_Fear, Overlap Count: 1
  Feature: Trend_Match_Fear, Overlap Count: 2
  Feature: Cor_Pred_Like, Overlap Count: 1
  Feature: Cor_Pred_Like_No_Match, Overlap Count: 1


Metric: BAS_D
  Feature: Mean_Rating0_No_Match_Happy, Overlap Count: 11
  Feature: Trend_No_Match_Sad, Overlap Count: 1
  Feature: Intercept_Match_F

### Bootstrapping with resampling in order to find robust cut-off against different data

In [17]:
epoch = 100
dict_bootstrap = {}


#dataframes = {}
for i in tqdm.tqdm(range(epoch),desc="Bootstraping"):
    sample_indices = np.random.choice(feature_scales.index, size=len(feature_scales), replace=True)
    sample_df = feature_scales.loc[sample_indices]
    all_dictionary = {}

    # Cluster the bootstrap sample
    for value in values:
    
        # Assuming feature_scales is your DataFrame with features as columns and metrics as rows
        dictionary_distribution = {}
    
        for metric in metrics_columns:
            dictionary_distribution[metric] = {}
    
            for feature in predictive_processing_columns:
                percentile = np.percentile(sample_df[feature], value)
                if value < 50:
                    above_values = sample_df[sample_df[feature] > percentile][metric]
                    below_values = sample_df[sample_df[feature] <= percentile][metric]
                else:
                    above_values = sample_df[sample_df[feature] >= percentile][metric]
                    below_values = sample_df[sample_df[feature] < percentile][metric]
                try:
                    u_stat, p_value = stats.mannwhitneyu(above_values, below_values, alternative='two-sided')
                    # Calculate means
                    high_mean = np.mean(above_values)
                    low_mean = np.mean(below_values)
        
                    dictionary_distribution[metric][feature] = {'high_mean': high_mean, 'size_high': len(above_values),
                                                                'low_mean': low_mean, 'size_low': len(below_values),
                                                                'p_value': p_value}
                except:
                    dictionary_distribution[metric][feature] = {'high_mean': 0, 'size_high': 0,
                                                                'low_mean': 0, 'size_low': 0,
                                                                'p_value': 1}
    
        all_dictionary[value] = dictionary_distribution
        df = pd.DataFrame.from_dict({(metric, feature): values
                                     for metric, features in dictionary_distribution.items()
                                     for feature, values in features.items()}, orient='index')
    
        # Add this DataFrame to the main dictionary of DataFrames
        #dataframes[value] = df
    
    dict_bootstrap[i] = all_dictionary

Bootstraping: 100%|██████████| 100/100 [1:18:26<00:00, 47.06s/it]


In [26]:
# Aggregate and compute means and variances
aggregate_stats = {value: {
    metric: defaultdict(lambda: {'p_values': [], 'high_means': [], 'low_means': [], 'size_highs': [], 'size_lows': []})
    for metric in metrics_columns} for value in values}
for i in range(epoch):
    for value in values:
        for metric, features in dict_bootstrap[i][value].items():
            for feature, data in features.items():
                agg = aggregate_stats[value][metric][feature]
                agg['p_values'].append(data['p_value'])
                agg['high_means'].append(data['high_mean'])
                agg['low_means'].append(data['low_mean'])
                agg['size_highs'].append(data['size_high'])
                agg['size_lows'].append(data['size_low'])

In [31]:
# Identify significant features
significant_features = {value: {} for value in values}
for value in values:
    for metric, features in aggregate_stats[value].items():
        significant_features[value][metric] = {}
        for feature, stats in features.items():
            mean_p = np.mean(stats['p_values'])
            var_p = np.var(stats['p_values'])
            mean_high_mean = np.mean(stats['high_means'])
            var_high_mean = np.var(stats['high_means'])
            mean_low_mean = np.mean(stats['low_means'])
            var_low_mean = np.var(stats['low_means'])
            mean_size_high = np.mean(stats['size_highs'])
            var_size_high = np.var(stats['size_highs'])
            mean_size_low = np.mean(stats['size_lows'])
            var_size_low = np.var(stats['size_lows'])
            if mean_p < 0.065:                
                significant_features[value][metric][feature] = {
                    'mean_p_value': mean_p,
                    'variance_p_value': var_p,
                    'mean_high_mean': mean_high_mean,
                    'variance_high_mean': var_high_mean,
                    'mean_low_mean': mean_low_mean,
                    'variance_low_mean': var_low_mean,
                    'mean_size_high': mean_size_high,
                    'variance_size_high': var_size_high,
                    'mean_size_low': mean_size_low,
                    'variance_size_low': var_size_low
                }


# Display results
for value in values:
    print(f"\nPercentile Value: {value}%")
    if value in significant_features and significant_features[value]:
        for metric, features in significant_features[value].items():
            print(f"Metric: {metric}")
            for feature, stats in features.items():
                print(f"  Feature: {feature}")
                print(f"    Mean p-value: {stats['mean_p_value']:.4f}, Variance p-value: {stats['variance_p_value']:.4f}")
                print(f"    Mean High Mean: {stats['mean_high_mean']:.2f}, Variance High Mean: {stats['variance_high_mean']:.2f}")
                print(f"    Mean Low Mean: {stats['mean_low_mean']:.2f}, Variance Low Mean: {stats['variance_low_mean']:.2f}")
                print(f"    Mean Size High: {stats['mean_size_high']:.2f}, Variance Size High: {stats['variance_size_high']:.2f}")
                print(f"    Mean Size Low: {stats['mean_size_low']:.2f}, Variance Size Low: {stats['variance_size_low']:.2f}")
    else:
        print("  No significant features at this percentile.")


Percentile Value: 5%
Metric: PA
  Feature: Mean_Rating0
    Mean p-value: 0.0605, Variance p-value: 0.0155
    Mean High Mean: 33.62, Variance High Mean: 0.32
    Mean Low Mean: 27.95, Variance Low Mean: 4.27
    Mean Size High: 139.61, Variance Size High: 2.42
    Mean Size Low: 9.39, Variance Size Low: 2.42
  Feature: Final_Value_No_Match_Sad
    Mean p-value: 0.0388, Variance p-value: 0.0102
    Mean High Mean: 33.63, Variance High Mean: 0.33
    Mean Low Mean: 27.49, Variance Low Mean: 3.68
    Mean Size High: 140.07, Variance Size High: 1.77
    Mean Size Low: 8.93, Variance Size Low: 1.77
Metric: NA.
Metric: ERQ_CR
Metric: ERQ_ES
Metric: UPPSP_NU
Metric: UPPSP_PU
  Feature: Trend_Match_Sad
    Mean p-value: 0.0547, Variance p-value: 0.0166
    Mean High Mean: 9.96, Variance High Mean: 0.04
    Mean Low Mean: 8.19, Variance Low Mean: 0.30
    Mean Size High: 140.20, Variance Size High: 1.02
    Mean Size Low: 8.80, Variance Size Low: 1.02
Metric: UPPSP_SS
Metric: UPPSP_PMD
Metric

### Removing Outliers

### Relation with 6 clusters PANASERQUPPSP

In [79]:
clusters_6_scales = pd.read_csv("subject_ids_with_clusters_all_cases_common_scales_6_clusters.csv")
clusters_6_scales = clusters_6_scales[["EPRIME_CODE","cluster"]]

In [80]:
df_features_clusters = pd.merge(left = predictive_processing_dataset,right=clusters_6_scales, left_on="Subject",right_on="EPRIME_CODE").drop(["EPRIME_CODE","Subject"],axis=1)
df_features_clusters["cluster"]=df_features_clusters["cluster"].astype('category')


In [81]:
from scipy.stats import chi2_contingency

all_dictionary = {}
results = []  # List to collect all results for DataFrame creation

predictive_processing_columns = df_features_clusters.columns.difference(['cluster'])

# Iterate over each percentile
for value in values:
    dictionary_distribution = {}

    # For each feature, calculate the percentile and bin data
    for feature in predictive_processing_columns:
        percentile = np.percentile(df_features_clusters[feature], value)
        # Binning data
        df_features_clusters['Bin'] = np.where(df_features_clusters[feature] >= percentile, 'Above', 'Below')

        # Create a contingency table
        contingency_table = pd.crosstab(df_features_clusters['cluster'], df_features_clusters['Bin'])

        # Perform the Chi-Square test
        chi2, p_value, _, _ = chi2_contingency(contingency_table)

        # Store the results
        dictionary_distribution[feature] = {
            'p_value': p_value
        }
        result_row = {
            'Percentile': value,
            'Feature': feature,
            'P-value': p_value
        }

 # Include counts for each cluster and bin, ensuring no KeyError
        for cluster in df_features_clusters['cluster'].unique():
            for bin in ['Above', 'Below']:
                column_name = f'Count_{cluster}_{bin}'
                result_row[column_name] = contingency_table.get(bin, {}).get(cluster, 0)
        results.append(result_row)


    # Store distributions for each percentile
    all_dictionary[value] = dictionary_distribution
    
results_df = pd.DataFrame(results)

# Print or analyze results
print(all_dictionary)


{5: {'Cor_Pred_Like': {'p_value': 0.13780170748923487}, 'Cor_Pred_Like_Match': {'p_value': 0.22886729532317926}, 'Cor_Pred_Like_Match_Fear': {'p_value': 0.8245382549842015}, 'Cor_Pred_Like_Match_Happy': {'p_value': 0.7994949367181868}, 'Cor_Pred_Like_Match_Sad': {'p_value': 0.5244699286291783}, 'Cor_Pred_Like_No_Match': {'p_value': 0.12373687438844883}, 'Cor_Pred_Like_No_Match_Fear': {'p_value': 0.5374006310486239}, 'Cor_Pred_Like_No_Match_Happy': {'p_value': 0.7092417842375415}, 'Cor_Pred_Like_No_Match_Sad': {'p_value': 0.7759676010703183}, 'Dif_Fear': {'p_value': 0.2517362987652178}, 'Dif_Happy': {'p_value': 0.8798613504430556}, 'Dif_Match': {'p_value': 0.7004748250276375}, 'Dif_Sad': {'p_value': 0.3250096643724522}, 'Final_Value_Match': {'p_value': 0.6222635973399865}, 'Final_Value_Match_Fear': {'p_value': 0.3078620368099997}, 'Final_Value_Match_Happy': {'p_value': 0.4977305449664473}, 'Final_Value_Match_Sad': {'p_value': 0.13002711761628452}, 'Final_Value_No_Match': {'p_value': 0.8

In [83]:
results_df[results_df["P-value"]<0.051]

,Percentile,Feature,P-value,Count_1_Above,Count_1_Below,Count_0_Above,Count_0_Below,Count_2_Above,Count_2_Below,Count_4_Above,Count_4_Below,Count_5_Above,Count_5_Below,Count_3_Above,Count_3_Below
20,5,Final_Value_No_Match_Sad,0.000997,34,0,27,2,28,0,40,3,7,0,5,3
32,5,Mean_Rating0_Match_Happy,0.044452,34,0,27,2,27,1,41,2,5,2,8,0
35,5,Mean_Rating0_No_Match_Fear,0.044616,34,0,28,1,28,0,43,0,6,1,8,0
66,10,Final_Value_No_Match_Sad,0.005718,33,1,26,3,26,2,39,4,6,1,4,4
112,15,Final_Value_No_Match_Sad,0.036353,32,2,24,5,24,4,39,4,6,1,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,80,Dif_Sad,0.028166,11,23,10,19,4,24,3,40,1,6,1,7
725,80,Mean_Rating0_No_Match_Fear,0.023713,1,33,11,18,9,19,11,32,1,6,2,6
731,80,Trend_Match_Sad,0.040034,2,32,7,22,10,18,6,37,2,5,3,5
747,85,Dif_Match,0.049548,11,23,6,23,4,24,4,39,0,7,0,8
